### Is a WOW station within the boundaries of the Netherlands?
Some notes on carrying out a `point-in-polygon` analysis, a relatively frequent GIS problem that might become complex quickly!

#### Introduction

Finding whether a point is within a polygon is a fairly common GIS problem that quickly becomes temporally costly if it needs to be done to large data collections. In this example three ways of tackling this operation are presented. Each of the approaches combines different Python libraries. The goal of this notebook is to illustrate why it is important to make sure that this operation is efficient by measuring the temporal cost of the three suggested approaches. For this purpose, we take 1 day of WOW data from the WOW Livestream (available in `./data-common/raw_wow_livestream/WOW_2023-09-01T00_00_00Z_2023-09-02T00_00_00Z.csv`) and we run the `point-in-polygon` operation for 100K of the observations, measuring the time it takes to run this operation for a large batch of data. 

The table below summarizes the temporal cost:

| Approach    | Temporal cost (for 100k) | Libraries/Functions used |
| :-------- | :------- | :------- |
| 1  | 200s    | fiona, shapely, pandas' iterrows()
| 2 | 194s     | fiona, shapely, pandas' apply()
| 3    | <1s    | geopandas' sjoin()
| 4  | 225s |   Python OGR (no iterrows())
| 5  | 249s |   Python OGR (with iterrows())





In [1]:
import time
import fiona
import shapely
import shapefile
import pandas as pd
import geopandas as gpd
from osgeo import osr, ogr
from shapely.geometry import (Point, shape)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [2]:
# Setting up some paths. Note that the available contour is "EPSG:4326", aka "WGS84", aka "latlon"
path_obs = r"./data-common/raw_wow_livestream/WOW_2023-09-01T00_00_00Z_2023-09-02T00_00_00Z.csv"
path_contour = r"./geodata/vector/NL_shapefile_WGS84/NL_WGS84.shp"

df_wow = pd.read_csv(path_obs, sep=",", header=0, parse_dates=["CreatedDateTime", "ReportEndDateTime", "LocalReportEndDateTime"])
gdf_wow = gpd.GeoDataFrame(df_wow, geometry=gpd.points_from_xy(df_wow.Longitude, df_wow.Latitude), crs="EPSG:4326")

print(df_wow.shape, gdf_wow.shape)

/tmp/ipykernel_50752/4020063117.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wow = pd.read_csv(path_obs, sep=",", header=0, parse_dates=["CreatedDateTime", "ReportEndDateTime", "LocalReportEndDateTime"])


(251285, 31) (251285, 32)


#### A lazy approach with `fiona`, `shapely` and `.iterrows()`

This one is a no-brainer. You open the `.shp` file, lazily iterate over the dataframe rows with `.iterrows()` while calculating the `.within()` operation. 

In [3]:
gdf_wow = gdf_wow.head(100000)
print("Number of features: ", gdf_wow.shape[0])
with fiona.open(path_contour) as shp:
    poly_contour = shape(shp[0]["geometry"])
    st = time.time()
    #Also, note .iterrows() is a temporally costly method!
    for idx, row in gdf_wow.iterrows(): 
        # print(idx, row["geometry"].within(poly_contour))
        row["geometry"].within(poly_contour)
    et = time.time()
    print("Ellapsed: {0} seconds".format(round(et-st, 2)))
      

Number of features:  100000
Ellapsed: 200.11 seconds


#### Another lazy approach with `fiona`, `shapely` and `.apply()`
I tried the same approach, but this time with `.apply()`, hoping it would have a better temporal cost due to the removal of `.iterrows()`, but not really. 

In [4]:
def point_in_poly(pt, poly):
    return pt.within(poly)

gdf_wow = gdf_wow.head(100000)
print("Number of features: ", gdf_wow.shape[0])
with fiona.open(path_contour) as shp:
    poly_contour = shape(shp[0]["geometry"])
    st = time.time()
    gdf_wow.apply(lambda row: point_in_poly(row['geometry'], poly_contour), axis=1)
    et = time.time()
    print("Ellapsed: {0} seconds".format(round(et-st, 2)))

Number of features:  100000
Ellapsed: 194.14 seconds


#### An approach using `geopandas`\' spatial join, `.sjoin()`
The fastest, but less explicit than the previous two. Requires thinking in database terms and joining tables. It's fast because `geopandas`' `.sjoin()` function has been improved substantially since it's first versions. 

In [5]:
gdf_nl = gpd.read_file(path_contour)
gdf_wow = gdf_wow.head(100000)

print("Number of features: ", gdf_wow.shape[0])
st = time.time()
gdf_sjoin = gpd.sjoin(gdf_wow, gdf_nl, how='left', op='within')
gdf_in = gdf_sjoin.query("index_right == 0")
gdf_out = gdf_sjoin.query("index_right != 0")
et = time.time()
print("Ellapsed: {0} seconds".format(round(et-st, 2)))

Number of features:  100000


/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Ellapsed: 0.23 seconds


#### A classic way of doing this operation: OGR/GDAL
Python bindings for C++ libraries, hence, not the most pythonic grammar! I was expecting more speed, but perhaps it would scale better for larger batches. 

In [6]:
gdf_wow = gdf_wow.head(100000)
print("Number of features: ", gdf_wow.shape[0])

# Reading the feature's geometry 'the old way'
driverName = "ESRI Shapefile"      # e.g.: GeoJSON, ESRI Shapefile
driver = ogr.GetDriverByName(driverName)
dataSource = driver.Open(path_contour, 0) # 0 means read-only. 1 means writeable.
layer = dataSource.GetLayer()
poly_contour = layer.GetNextFeature()
poly_geom = poly_contour.GetGeometryRef()
featureCount = layer.GetFeatureCount()

st = time.time()
# Unpack the dataframe to avoid using iterrows
l = gdf_wow[["Longitude", "Latitude"]].values.tolist()

for item in l:
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(item[0], item[1])
    point.Within(poly_geom)
et = time.time()
print("Ellapsed (no iterrows()): {0} seconds".format(round(et-st, 2)))    

st = time.time()
for idx, row in gdf_wow.iterrows(): 
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(row["Longitude"], row["Latitude"])
    point.Within(poly_geom)
et = time.time()
print("Ellapsed (with iterrows()): {0} seconds".format(round(et-st, 2)))

Number of features:  100000
Ellapsed (no iterrows()): 225.01 seconds
Ellapsed (with iterrows()): 249.23 seconds
